## TCP

In [ ]:
# TCP 
import socket
import time
TCP_IP = '192.168.0.152'
TCP_PORT = 5066

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    print(f"Connecting to {TCP_IP}:{TCP_PORT}")
    s.connect((TCP_IP, TCP_PORT))
except socket.error as msg:
    print(f"Failed to connect: {msg}")

print(f"Successfully connected to {TCP_IP}:{TCP_PORT}")

while True:
        
    s.send('Hello, World!'.encode('utf-8'))
    time.sleep(1)
    
s.close()



Connecting to 192.168.0.152:5066
Successfully connected to 192.168.0.152:5066


ConnectionAbortedError: [WinError 10053] 連線已被您主機上的軟體中止。

## UDP

## Sensor

In [1]:
import serial

ser = serial.Serial('COM6', 9600)  

while True:
    if ser.in_waiting > 0:
        data = ser.readline().decode('utf-8').strip()
        print(f"Received data: {data}")


Received data: Ranging(mm) = 275, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 273, status = 0x0
Received data: Ranging(mm) = 273, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 273, status = 0x0
Received data: Ranging(mm) = 273, status = 0x0
Received data: Ranging(mm) = 275, status = 0x0
Received data: Ranging(mm) = 273, status = 0x0
Received data: Ranging(mm) = 273, status = 0x0
Received data: Ranging(mm) = 273, status = 0x0
Received data: Ranging(mm) = 273, status = 0x0
Received data: Ranging(mm) = 275, status = 0x0
Received data: Ranging(mm) = 275, status = 0x0
Received data: Ranging(mm) = 275, status = 0x0
Received data: Ranging(mm) = 275, status = 0x0
Received data: Ranging(mm) = 275, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data

KeyboardInterrupt: 

## total test

In [ ]:
import socket
import time
import serial

TCP_IP = '192.168.0.152'
TCP_PORT = 5066

ser = serial.Serial('COM6', 9600)

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    print(f"Connecting to {TCP_IP}:{TCP_PORT}")
    s.connect((TCP_IP, TCP_PORT))
except socket.error as msg:
    print(f"Failed to connect: {msg}")

print(f"Successfully connected to {TCP_IP}:{TCP_PORT}")

while True:
    if ser.in_waiting > 0:
        data = ser.readline().decode('utf-8').strip()
        print(f"Received data: {data}")
        s.send(data.encode('utf-8'))
    time.sleep(1)

Connecting to 192.168.0.152:5066
Successfully connected to 192.168.0.152:5066
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 275, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 275, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0
Received data: Ranging(mm) = 274, status = 0x0


KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_BRIGHTNESS, 100)
cap.set(cv2.CAP_PROP_CONTRAST, 15)
cap.set(cv2.CAP_PROP_SATURATION, 36)
cap.set(cv2.CAP_PROP_HUE, 0)
cap.set(cv2.CAP_PROP_EXPOSURE, -6)


def find_rect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    cv2.imshow('Blur', blur)
    a, thresh = cv2.threshold(blur, 200, 255, cv2.THRESH_BINARY_INV)
    cv2.imshow('Thresh', thresh)
    erode = cv2.erode(thresh, (7, 7), iterations=5)
    # cv2.imshow('Erode', erode)
    dilate = cv2.dilate(erode, (7, 7), iterations=5)
    # cv2.imshow('Dilate', dilate)
    contours, _ = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    cnt_max = max(contours, key=cv2.contourArea)
    return cnt_max

qrcode = cv2.QRCodeDetector()  

while True:
    ret, frame = cap.read()
    if not ret:
        print("Cannot receive frame")
        break
    img = cv2.resize(frame,(720,420))
    ok, data, bbox, rectified = qrcode.detectAndDecodeMulti(img) 
    
    cnt_max = find_rect(img)
    if cnt_max is not None:
        rect = cv2.minAreaRect(cnt_max)
        
        angle = rect[2]
        
        if angle < -45:
            angle = 90 + angle

        (h, w) = img.shape[:2]
        center = (w // 2, h // 2)

        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        
        box = np.int32(cv2.boxPoints(rect))
        
        pts = np.int32(cv2.transform(np.array([box]), M))[0]

        y_min = min(pts[0][0], pts[1][0], pts[2][0], pts[3][0])
        y_max = max(pts[0][0], pts[1][0], pts[2][0], pts[3][0])
        x_min = min(pts[0][1], pts[1][1], pts[2][1], pts[3][1])
        x_max = max(pts[0][1], pts[1][1], pts[2][1], pts[3][1])
        
        # print('x length:', x_max-x_min, ', y length:', y_max-y_min)
        print('x length:', x_max-x_min, ', y length:', y_max-y_min)
        
        cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
    
    
    cv2.imshow('Final', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


x length: 214 , y length: 376
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 213 , y length: 376
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 377 , y length: 214
x length: 214 , y length: 377
x length: 

In [ ]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image

def find_rect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    # cv2.imshow('Blur', blur)
    a, thresh = cv2.threshold(blur, 200, 255, cv2.THRESH_BINARY_INV)
    # cv2.imshow('Thresh', thresh)
    erode = cv2.erode(thresh, (7, 7), iterations=5)
    # cv2.imshow('Erode', erode)
    dilate = cv2.dilate(erode, (7, 7), iterations=5)
    # cv2.imshow('Dilate', dilate)
    contours, _ = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    cnt_max = max(contours, key=cv2.contourArea)
    return cnt_max

def snap_picture(cap,qrcode):
    
    ret, frame = cap.read()
    if not ret:
        print("Cannot receive frame")
        return None
    img = cv2.resize(frame,(720,420))
    ok, data, bbox, rectified = qrcode.detectAndDecodeMulti(img) 
    
    if not ok:
        input("QR code not detected. Please enter the package name:")
        
    cnt_max = find_rect(img)
    
    if cnt_max is not None:
        rect = cv2.minAreaRect(cnt_max)
        angle = rect[2]
        if angle < -45:
            angle = 90 + angle
        
        (h, w) = img.shape[:2]
        center = (w // 2, h // 2)

        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        
        box = np.int32(cv2.boxPoints(rect))
        
        pts = np.int32(cv2.transform(np.array([box]), M))[0]

        y_min = min(pts[0][0], pts[1][0], pts[2][0], pts[3][0])
        y_max = max(pts[0][0], pts[1][0], pts[2][0], pts[3][0])
        x_min = min(pts[0][1], pts[1][1], pts[2][1], pts[3][1])
        x_max = max(pts[0][1], pts[1][1], pts[2][1], pts[3][1])
        
        print('data:', data, 'x length:', x_max-x_min, ', y length:', y_max-y_min)
        cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
    
    return img


cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_BRIGHTNESS, 100)
cap.set(cv2.CAP_PROP_CONTRAST, 15)
cap.set(cv2.CAP_PROP_SATURATION, 36)
cap.set(cv2.CAP_PROP_HUE, 0)
cap.set(cv2.CAP_PROP_EXPOSURE, -6)

qrcode = cv2.QRCodeDetector()  

last_time = time.time()
while True:
    if time.time() - last_time > 5:
        img = snap_picture(cap,qrcode)
        last_time = time.time()
    
    # k = cv2.waitKey(1)
    # if k == ord('q'):
    #     break
    # elif k == ord('s'):
    #     img = snap_picture(cap,qrcode)
        


KeyboardInterrupt: 

In [ ]:
import socket
import time
import serial
import cv2
import threading

# TCP/IP
TCP_IP = '192.168.0.152'
TCP_PORT = 5066
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    print(f"Connecting to {TCP_IP}:{TCP_PORT}")
    s.connect((TCP_IP, TCP_PORT))
except socket.error as msg:
    print(f"Failed to connect: {msg}")

print(f"Successfully connected to {TCP_IP}:{TCP_PORT}")

# Teensy
teensy_1 = serial.Serial('/dev/ttyACM0', 9600, timeout=1)
teensy_2 = serial.Serial('/dev/ttyACM1', 9600, timeout=1)

if teensy_1.is_open and teensy_2.is_open:
    print("Successfully connected to Teensy devices!")
    
# Camera
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_BRIGHTNESS, 100)
cap.set(cv2.CAP_PROP_CONTRAST, 15)
cap.set(cv2.CAP_PROP_SATURATION, 36)
cap.set(cv2.CAP_PROP_HUE, 0)
cap.set(cv2.CAP_PROP_EXPOSURE, -6)

def find_rect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    # cv2.imshow('Blur', blur)
    a, thresh = cv2.threshold(blur, 150, 255, cv2.THRESH_BINARY_INV)
    # cv2.imshow('Thresh', thresh)
    erode = cv2.erode(thresh, (7, 7), iterations=5)
    # cv2.imshow('Erode', erode)
    dilate = cv2.dilate(erode, (7, 7), iterations=5)
    # cv2.imshow('Dilate', dilate)
    contours, _ = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    cnt_max = max(contours, key=cv2.contourArea)
    return cnt_max

# QR Code
qrcode = cv2.QRCodeDetector()

# variable num
sensor1_num = 0
sensor2_num = 0
sensor3_num = 0
read_time = time.time()
send_time = time.time()

while True:
    if time.time()-read_time >= 0.03:
        if teensy_1.in_waiting > 0:
            data_1 = teensy_1.readline().decode('utf-8').strip()
            # print(f"Teensy 1 Data: {data_1}")
            data_1_sp = data_1.split(',')
        
            if len(data_1_sp) == 1:
                sensor3_num = data_1_sp[0].split(' ')[2]
            elif len(data_1_sp) == 2:
                sensor1_num = data_1_sp[0].split(' ')[2]
                sensor2_num = data_1_sp[1].split(' ')[2]
            else:
                print('Teensy 1 Error')
            
        if teensy_2.in_waiting > 0:
            data_2 = teensy_2.readline().decode('utf-8').strip()
            # print(f"Teensy 2 Data: {data_2}")
            data_2_sp = data_2.split(',')
            
            if len(data_2_sp) == 1:
                sensor3_num = data_2_sp[0].split(' ')[2]
            elif len(data_2_sp) == 2:
                sensor1_num = data_2_sp[0].split(' ')[2]
                sensor2_num = data_2_sp[1].split(' ')[2]
            else:
                print('Teensy 2 Error')
                
        print(f'Need sent Sensor1: {sensor1_num}, Sensor2: {sensor2_num}, Sensor3: {sensor3_num}')
        read_time = time.time()
    
    if time.time() - send_time >= 0.2:
        send_data = f'Sensor1: {sensor1_num},Sensor2: {sensor2_num},Sensor3: {sensor3_num}'
        s.send(send_data.encode('utf-8'))
        send_time = time.time()


In [ ]:
import socket
import time
import serial
import threading
import cv2
import numpy as np

# TCP/IP
TCP_IP = '192.168.0.152'
TCP_PORT = 5066
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    print(f"Connecting to {TCP_IP}:{TCP_PORT}")
    s.connect((TCP_IP, TCP_PORT))
except socket.error as msg:
    print(f"Failed to connect: {msg}")
    
print(f"Successfully connected to {TCP_IP}:{TCP_PORT}")
# =============================================================================

# Teensy
teensy_1 = serial.Serial('/dev/ttyACM0', 9600, timeout=1)
teensy_2 = serial.Serial('/dev/ttyACM1', 9600, timeout=1)

if teensy_1.is_open and teensy_2.is_open:
    print("Successfully connected to Teensy devices!")
# =============================================================================
    
# Camera
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_BRIGHTNESS, 100)
cap.set(cv2.CAP_PROP_CONTRAST, 15)
cap.set(cv2.CAP_PROP_SATURATION, 36)
cap.set(cv2.CAP_PROP_HUE, 0)
cap.set(cv2.CAP_PROP_EXPOSURE, -6)

def find_rect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    # cv2.imshow('Blur', blur)
    a, thresh = cv2.threshold(blur, 220, 255, cv2.THRESH_BINARY_INV)
    # cv2.imshow('Thresh', thresh)
    erode = cv2.erode(thresh, (7, 7), iterations=5)
    # cv2.imshow('Erode', erode)
    dilate = cv2.dilate(erode, (7, 7), iterations=5)
    # cv2.imshow('Dilate', dilate)
    contours, _ = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    cnt_max = max(contours, key=cv2.contourArea)
    return cnt_max

# QR Code
qrcode = cv2.QRCodeDetector()

def snap_picture(cap,qrcode):
    
    ret, frame = cap.read()
    if not ret:
        print("Cannot receive frame")
        return None
    img = cv2.resize(frame,(720,420))
    ok, data, bbox, rectified = qrcode.detectAndDecodeMulti(img) 
    
    if not ok:
        data = input("QR code not detected. Please enter the package name:")
        
    cnt_max = find_rect(img)
    
    if cnt_max is not None:
        rect = cv2.minAreaRect(cnt_max)
        angle = rect[2]
        if angle < -45:
            angle = 90 + angle
        
        (h, w) = img.shape[:2]
        center = (w // 2, h // 2)

        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        
        box = np.int32(cv2.boxPoints(rect))
        
        pts = np.int32(cv2.transform(np.array([box]), M))[0]

        y_min = min(pts[0][0], pts[1][0], pts[2][0], pts[3][0])
        y_max = max(pts[0][0], pts[1][0], pts[2][0], pts[3][0])
        x_min = min(pts[0][1], pts[1][1], pts[2][1], pts[3][1])
        x_max = max(pts[0][1], pts[1][1], pts[2][1], pts[3][1])
        
        # print('data:', data, 'x length:', x_max-x_min, ', y length:', y_max-y_min)
        cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
        
    return img, data, x_max-x_min, y_max-y_min

# =============================================================================

    
sensor1_num = 0
sensor2_num = 0
sensor3_num = 0


def Read_Sensor(teensy1, teensy2, delay):
    global sensor1_num
    global sensor2_num
    global sensor3_num
    
    while True:
        if teensy1.in_waiting > 0:
            data_1 = teensy1.readline().decode('utf-8').strip()
            data_1_sp = data_1.split(',')
        
            if len(data_1_sp) == 1:
                sensor3_num = data_1_sp[0].split(' ')[2]
            elif len(data_1_sp) == 2:
                sensor1_num = data_1_sp[0].split(' ')[2]
                sensor2_num = data_1_sp[1].split(' ')[2]
            else:
                print('Teensy 1 Error')
            
        if teensy2.in_waiting > 0:
            data_2 = teensy2.readline().decode('utf-8').strip()
            data_2_sp = data_2.split(',')
            
            if len(data_2_sp) == 1:
                sensor3_num = data_2_sp[0].split(' ')[2]
            elif len(data_2_sp) == 2:
                sensor1_num = data_2_sp[0].split(' ')[2]
                sensor2_num = data_2_sp[1].split(' ')[2]
            else:
                print('Teensy 2 Error')
                
        time.sleep(delay)

readSensor = threading.Thread(target=Read_Sensor, args=(teensy_1, teensy_2, 0.03))
readSensor.daemon = True
readSensor.start()


def Send_Data(cap, qrcode, delay):
    global sensor1_num
    global sensor2_num
    take_picture = False
    
    def set_take_picture_true():
        nonlocal take_picture
        take_picture = True
    
    while True:
        send_data = f'Sensor1: {sensor1_num},Sensor2: {sensor2_num}'
        s.send(send_data.encode('utf-8'))
        box_high = 600 - int(sensor3_num)
        print(box_high)
        
        if int(sensor3_num) !=0 and box_high >= 50 and box_high <= 300:
            
            if not take_picture:
                threading.Timer(2.0, set_take_picture_true).start()
                
            if take_picture:
                img, data, x_length, y_length = snap_picture(cap,qrcode)
                send_data = f'Package_Name: {data},X_length: {x_length},Y_length: {y_length},Sensor3: {sensor3_num}'
                s.send(send_data.encode('utf-8'))
                cv2.imshow('Final', img)
                take_picture = False
                
        time.sleep(delay)

send_Data = threading.Thread(target=Send_Data, args=(cap,qrcode,0.2))
send_Data.start()

def recvTCP():
    while True:
        msg_TCP= s.recv(1024)
        if msg_TCP is not None:
            print('recv:'+ msg_TCP.decode())
        
recv_Data = threading.Thread(target=recvTCP)
recv_Data.start()


In [2]:
import toml

output_file = ".streamlit/secrets.toml"

with open("./key.json") as json_file:
    json_text = json_file.read()

config = {"textkey": json_text}
toml_config = toml.dumps(config)

with open(output_file, "w") as target:
    target.write(toml_config)

In [14]:
from google.cloud import firestore

db = firestore.Client.from_service_account_json('./key.json')

dict_data = {}
# Create a reference to the Google post.
doc_ref = db.collection("IOT").document("PackageName")

doc_ref.set(dict_data)

update_time {
  seconds: 1734360984
  nanos: 590297000
}

In [16]:
datatest = "No.testing1:0,No.testing2:1,"

# convert string to dictionary
# dict_data = {x.split(':')[0]: int(x.split(':')[1]) for x in datatest.split(',')}
dict_data = {}

for x in datatest.split(','):
    if x == '':
        continue
    key = x.split(':')[0]
    value = int(x.split(':')[1])
    dict_data[key] = value

print(dict_data)


# dict_data = {"Package No. testing": 0,}

{'No.testing1': 0, 'No.testing2': 1}


In [ ]:
import pandas as pd

# 原始字典
dict_data = {"No.1": 5}

# 創建 DataFrame
df = pd.DataFrame(list(dict_data.items()), columns=["Package NO.", "Number"])

# 重命名列
df.columns = df.iloc[0]  # 將第一行設為列名
df = df[1:]  # 刪除第一行

df.head()

,"(Package NO., Number)","(No.1, 5)"


In [27]:
# dict add
dict_data["No.2"] = 6
dict_data


{'Package NO.': 'Number', 'No.1': 5, 'No.2': 6}

In [ ]:
def Send_Data(cap, qrcode, delay):
    global sensor1_num
    global sensor2_num
    take_picture = False
    
    def set_take_picture_true():
        nonlocal take_picture
        box_high = 560 - int(sensor3_num)
        
        if int(sensor3_num) != 0 and 50 <= box_high <= 300:
            take_picture = True
        else:
            threading.Timer(2.0, set_take_picture_true).start()
    
    while True:
        ret, frame = cap.read()
        send_data = f'Sensor1: {sensor1_num},Sensor2: {sensor2_num}'
        s.send(send_data.encode('utf-8'))
        box_high = 560 - int(sensor3_num)
        print(box_high)

        if int(sensor3_num) !=0 and box_high >= 50 and box_high <= 300:
            
            if not take_picture:
                threading.Timer(2.0, set_take_picture_true).start()
                
            if take_picture:
                img, data, x_length, y_length = snap_picture(ret,frame,qrcode)
                send_data = f'Package_Name: {data},X_length: {x_length},Y_length: {y_length},Sensor3: {box_high}'
                s.send(send_data.encode('utf-8'))
                cv2.imwrite(f'/home/kialas/IOT Case/picture.png', img)
                take_picture = False
                
        time.sleep(delay)